In [2]:
import json
import pathlib
from pprint import pprint
from collections import defaultdict

json_filename = 'installers.json'

json_file = pathlib.Path(json_filename)

In [3]:
with open(json_file, 'r') as f:
    installers_dicts = json.load(f)

In [4]:
installer_id = '2020.02_py3.8_win-64'

In [5]:
all_pkg_set = set()
for d in installers_dicts:
    if 'pkgs' in installers_dicts[d]:
        all_pkg_set.update(installers_dicts[d]['pkgs'])

In [6]:
# for pkg in installers_dicts[installer_id]['pkgs'].keys():
#     pprint(installers_dicts[installer_id]['pkgs'][pkg].keys())

    # pkgs_dict[pkg] = {}
    # pkgs_dict[pkg][]

In [6]:
len(all_pkg_set)

881

In [10]:
pkgs_dict = {p: defaultdict(lambda: defaultdict(lambda: defaultdict(str))) for p in all_pkg_set}



In [11]:
for d in installers_dicts:
    if 'pkgs' in installers_dicts[d]:
        for p in installers_dicts[d]['pkgs']:
            pkgs_dict[p]['pkg_url'] = installers_dicts[d]['pkgs'][p]['link']
            pkgs_dict[p]['pkg_summary'] = installers_dicts[d]['pkgs'][p]['summary']
            pkgs_dict[p]['installers'][d]['installer_url'] = installers_dicts[d]['url']
            pkgs_dict[p]['installers'][d]['anaconda'] = installers_dicts[d]['anaconda']
            pkgs_dict[p]['installers'][d]['python'] = installers_dicts[d]['python']
            pkgs_dict[p]['installers'][d]['pkg_version'] = installers_dicts[d]['pkgs'][p]['version']
            pkgs_dict[p]['installers'][d]['pkg_included'] = installers_dicts[d]['pkgs'][p]['included']


In [12]:
pkgs_dict['beautifulsoup4']

defaultdict(<function __main__.<dictcomp>.<lambda>()>,
            {'pkg_url': 'http://www.crummy.com/software/BeautifulSoup/',
             'pkg_summary': 'Python library designed for screen-scraping / MIT',
             'installers': defaultdict(<function __main__.<dictcomp>.<lambda>.<locals>.<lambda>()>,
                         {'2.4.0_py27': defaultdict(str,
                                      {'installer_url': 'https://docs.anaconda.com/anaconda/packages/old-pkg-lists/2.4.0/py27/',
                                       'anaconda': '2.4.0',
                                       'python': '2.7',
                                       'pkg_version': '4.4.1',
                                       'pkg_included': True}),
                          '2.4.0_py34': defaultdict(str,
                                      {'installer_url': 'https://docs.anaconda.com/anaconda/packages/old-pkg-lists/2.4.0/py34/',
                                       'anaconda': '2.4.0',
                 